In [8]:
# importing modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from modules.library import documentRetrieval as dr
from modules.library import queryExp as qe
from modules.library.postgresql import PostgresQL
pg = PostgresQL() 


In [2]:
# importing word vectors
from gensim.models import KeyedVectors
wiki_en_align = './../data/fasttext/wiki.en.align.vec' #'../../data/fasttext/wiki.en.align.vec'
# get fasttext wiki embeddings for english
wv_wiki_en = KeyedVectors.load_word2vec_format(wiki_en_align)
print('english words {}'.format(len(list(wv_wiki_en.vocab.keys()))))

C:\Users\sarab\Miniconda3\envs\envirolens\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


english words 2519370


In [3]:
# prepare stopword list
from nltk.corpus   import stopwords
from nltk.tokenize import word_tokenize
stopWords = stopwords.words('english')

In [9]:

pg.connect(database="eurlex_environment_only", user="postgres", password="dbpass") #"eurlex_env_only" "solata.2018"

In [10]:
def db_query(query_words):
    """ Vrne seznam dokumentov iz baze, ki vsebujejo vsaj eno od besed iz seznama(list) query_words"""
    output = '|'.join(query_words)
    SQL = """
        SELECT * FROM documents
        WHERE document_text @@ to_tsquery(""" + '\''+ output + '\'' + """);"""
    documents = pg.execute(SQL)
    return(documents)

In [12]:
tokenized = qe.tokenize('water pollution underground', stopWords)
ext = qe.extend_tokens(tokenized, wv_wiki_en)
candidates = qe.pre_retrieval_KNN('water pollution underground', 5, wv_wiki_en, 10, stopWords,extension=False)

In [13]:
docs = db_query(tokenized) #treba poizvedbo spravit v tako obliko da jo score function sprejme + preveri
# kam v score fn. vteknes candidate, ext + premisli kaj od tega das v db_query (I guess vse)

In [14]:
print(len(docs))

15845


In [15]:
docs[0]

{'document_id': 54624,
 'document_celex_num': '52013DC0319',
 'document_title': 'COMMUNICATION FROM THE COMMISSION TO THE COUNCIL concerning a consultation on Fishing Opportunities for 2014',
 'document_author': 'European Commission',
 'document_form': 'Communication',
 'document_date': datetime.date(2013, 5, 30),
 'document_text': '|52013DC0319COMMUNICATION FROM THE COMMISSION TO THE COUNCIL concerning a consultation on Fishing Opportunities for 2014 /* COM/2013/0319 final */COMMUNICATION FROM THE COMMISSION TOTHE COUNCILconcerning a consultation on FishingOpportunities for 20141.\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 IntroductionThis document sets out the initialintentions of the European Commission concerning the fixing of total allowablecatches (TACs) and effort levels in European waters and for European fishers in2014. Member States and Regional Advisory Councils (RACs) are invited toprovide their views to the Commission for consideration before a final decisionis taken on the C

In [7]:
score_sum = dr.probability_score(tokens,texts, probability_function,n, *args)
probab_score_sum(test+ext+candidates,tokenized_docs,texts)

NameError: name 'probab_score_sum' is not defined

In [ ]:
dr.probability_score(tokens,texts, probability_function,n, *args)
    # final function, takes also probability_function probability_sum_weight, but doesnt give final result (used in probability_score_sum_weights)
#     """Assigns score to documents based on probability_function metric.
#     Args:
#         tokens (list): List of tokens (tokenized query). If needed also extension (extension by summation of 2 consecutive words).
#         texts (dict):  Keys represent document ids, values are document text. 
#         probability_function (function): Metric function that calculates document relavance. Functions: probability_multiply, probability_sum. Require only first 4 arguments.
#         n (int): Number of returned tuples, sorted by highest scores.
#         top_expansion (list): List of expanded words. Usually candidates (kNN expansion).
#         alpha (float): Number between 0 and 1. Weight that emphasizes the difference between original query words and expansions. 
#                        For alpha 0.5 all words have same weights (but not same values!), for alpha 1 expansion words have value 0. 
#                        For alpha -1 values equal to cosine similarity to query words. 
#         wv (Word2VecKeyedVectors): Word embeddings.
#     Returns:
#         document_probability (list): Tuples of document ids and scores that measure document relavance. Returns n tuples with highest score.
#     """

In [ ]:
dr.probability_score_sum_weights(original_tokens, top_expansion, texts,n, alpha, wv)
    # final fuction
#     """As probability_score only weighted.
#         Args:
#         original_tokens(list): List of strings. Tokenized original query. Usually also extension (extension by summation of 2 consecutive words)
#         top_expansion (list): List of expanded words. Usually candidates (kNN expansion).
#         texts (dict):  Keys represent document ids, values are document text.
#         n (int): Number of returned tuples, sorted by highest scores.
#         alpha (float): Number between 0 and 1. Weight that emphasizes the difference between original query words and expansions. 
#                        For alpha 0.5 all words have same weights (but not same values!), for alpha 1 expansion words have value 0. 
#                        For alpha -1 values equal to cosine similarity to query words. 
#         wv (Word2VecKeyedVectors): Word embeddings.
#     Returns:
#         document_score (list): Tuples of document ids and scores that measure document relavance. Returns n tuples with highest score.
#     """

In [ ]:
dr.tfidf_score(tokens, texts, tfidf_function,n, *args)
#     #final function
#     """Assigns score to documents based on tfidf_function metric.
#     Args:
#         tokens (list): List of tokens (tokenized query). If needed also extension (extension by summation of 2 consecutive words).
#         texts (dict):  Keys represent document ids, values are document text. 
#         probability_function (function): Metric function that calculates document relavance. Functions: tfidf_sum; require only first 4 arguments, tfidf_sum_weight; require all arguments.
#         n (int): Number of returned tuples, sorted by highest scores.
#         top_expansion (list): List of expanded words. Usually candidates (kNN expansion).
#         alpha (float): Number between 0 and 1. Weight that emphasizes the difference between original query words and expansions. 
#                        For alpha 0.5 all words have same weights (but not same values!), for alpha 1 expansion words have value 0. 
#                        For alpha -1 values equal to cosine similarity to query words. 
#         wv (Word2VecKeyedVectors): Word embeddings.
#     Returns:
#         document_probability (list): Tuples of document ids and scores that measure document relavance. Returns n tuples with highest score.
#         not_appear (list): List of words that did not occure in any document.
#     """